In [ ]:
using DataFrames, Plots, StatsBase, JuMP, Statistics, Gurobi
using DataFrames, CSV, LinearAlgebra, Random

## Functions to perform linear regression using an optimal uncertainty set; functions return the optimal $\beta$

In [ ]:
function frobenius_one(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, abs_beta[1:p])

    @variable(mod, b)
    @variable(mod, abs_Ter1[1:n]>=0)
    
    @constraint(mod, -beta .<= abs_beta)
    @constraint(mod, beta .<= abs_beta)
    
    @constraint(mod, [i=1:p], abs_beta[i] <= b) # b = max of abs. value of elements of beta, which is the l-infinity norm
    
    @constraint(mod, -(y - X*beta) .<= abs_Ter1) # abs_Ter1 = abs(y - X*beta)
    @constraint(mod, (y - X*beta) .<= abs_Ter1)
    
    @objective(mod, Min, sum(abs_Ter1) + ρ*b)
    
    solve(mod)
    return(getvalue(beta))
    
end

In [ ]:
function frobenius_two(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, Ter1>=0)
    @variable(mod, Ter2>=0)
    
    @constraint(mod, Ter1 >= norm(y - X*beta)) # norm returns the l-2 norm
    @constraint(mod, Ter2 >= ρ*norm(beta))
    
    @objective(mod, Min, Ter1 + Ter2)
    
    solve(mod)
    return(getvalue(beta))
end

In [ ]:
function frobenius_infinity(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, abs_beta[1:p])
    @variable(mod, max_Ter1)

    @variable(mod, abs_Ter1[1:n]>=0)
    
    @constraint(mod, -beta .<= abs_beta) # abs_beta is the absolute value of the beta vector
    @constraint(mod, beta .<= abs_beta)
    
    @constraint(mod, -(y - X*beta) .<= abs_Ter1) # abs_Ter1 = abs(y - X*beta)
    @constraint(mod, (y - X*beta) .<= abs_Ter1)
    
    @constraint(mod, [i=1:n], abs_Ter1[i] <= max_Ter1) # max_Ter1 is the l-infinity norm of the vector (y-X*beta)
    
    @objective(mod, Min, max_Ter1 + ρ*sum(abs_beta))
    
    solve(mod)
    return(getvalue(beta))
    
end

In [ ]:
function induced_one_two(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, abs_Ter1[1:n]>=0)
    @variable(mod, Ter2>=0)
    
    @constraint(mod, -(y - X*beta) .<= abs_Ter1) # abs_Ter1 = abs(y - X*beta)
    @constraint(mod, (y - X*beta) .<= abs_Ter1)
    
    @constraint(mod, Ter2 >= ρ*norm(beta)) # Ter2 is ρ times the l-2 norm of beta
    
    @objective(mod, Min, sum(abs_Ter1) + Ter2)
    
    solve(mod)
    return(getvalue(beta)) 
end

In [ ]:
function induced_two_one(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, Ter1>=0)
    @variable(mod, abs_beta[1:p])
    
    @constraint(mod, Ter1 >= norm(y - X*beta)) # Ter1 is the l-2 norm of (y - X*beta)
    
    @constraint(mod, -beta .<= abs_beta)
    @constraint(mod, beta .<= abs_beta)
    
    @objective(mod, Min, Ter1 + ρ*sum(abs_beta))
    
    solve(mod)
    return(getvalue(beta))
end

In [ ]:
function induced_two_infinity(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, Ter1>=0)
    @variable(mod, abs_beta[1:p])
    @variable(mod, b)
    
    @constraint(mod, Ter1 >= norm(y - X*beta)) # Ter1 is the l-2 norm of (y - X*beta)
    
    @constraint(mod, -beta .<= abs_beta)
    @constraint(mod, beta .<= abs_beta)
    
    @constraint(mod, [i=1:p], abs_beta[i] <= b) # b = max of abs. value of elements of beta, which is the l-infinity norm
    
    @objective(mod, Min, Ter1 + ρ*b)
    
    solve(mod)
    return(getvalue(beta))
end

In [ ]:
function induced_infinity_two(X, y, ρ)
    mod = Model(solver=GurobiSolver(OutputFlag=0, Quad=1, PSDTol=1e-3))
    
    n = size(X)[1]
    p = size(X)[2]
    
    @variable(mod, beta[1:p])
    @variable(mod, max_Ter1)
    @variable(mod, abs_Ter1[1:n]>=0)
    @variable(mod, Ter2>=0)
    
    @constraint(mod, -(y - X*beta) .<= abs_Ter1) # abs_Ter1 = abs(y - X*beta)
    @constraint(mod, (y - X*beta) .<= abs_Ter1)
    
    @constraint(mod, [i=1:n], abs_Ter1[i] <= max_Ter1) # max_Ter1 is l-infinity norm of (y - X*beta)
    
    @constraint(mod, Ter2 >= ρ*norm(beta)) # Ter2 is ρ times the l-2 norm of beta
    
    @objective(mod, Min, max_Ter1 + Ter2)
    
    solve(mod)
    return(getvalue(beta))
    
end

### Function to split a dataset into training, validation, and testing sets

In [ ]:
function splitter(X_data, Y_data, percent_train, percent_valid)
    # Concatenate X and Y data
    data = hcat(Y_data, X_data)
    
    # Shuffle rows of X and Y data combined
    data = data[shuffle(1:end), :]
    
    # Split X and Y data
    Y_shuffled = data[:, end]
    X_shuffled = data[:, 1:end-1]
    
    # Get indices for splitting your data
    n = size(data)[1]
    train_ind = Int(round(percent_train*n))
    valid_ind = train_ind + Int(round(percent_valid*n))
    test_ind = n
    
    # Split X data based on indices
    X_train_data = X_shuffled[1:train_ind, :]
    X_valid_data = X_shuffled[train_ind+1:valid_ind, :]
    X_test_data = X_shuffled[valid_ind+1:n, :]
    
    # Split Y data based on indices
    Y_train_data = Y_shuffled[1:train_ind, :]
    Y_valid_data = Y_shuffled[train_ind+1:valid_ind, :]
    Y_test_data = Y_shuffled[valid_ind+1:n, :]
    
    #Now we normalize features!
    
    #Center features
    for i in 1:size(X_train_data,2)  
        X_train_data[:,i] = X_train_data[:,i] .- mean(X_train_data[:,i]);
        X_valid_data[:,i] = X_valid_data[:,i] .- mean(X_train_data[:,i]);
        X_test_data[:,i] = X_test_data[:,i] .- mean(X_train_data[:,i]);
    end
    
    #Calculate the std for each feature
    num_dims = size(X_train_data,2)
    std_feat = zeros(num_dims)
    for i in 1:num_dims
      std_feat[i] = std(X_train_data[:,i]);
    end
    
    #scale the features
    for i in 1:num_dims
        X_train_data[:,i]=X_train_data[:,i] ./std_feat[i]
        X_valid_data[:,i] = X_valid_data[:,i] ./ std_feat[i];
        X_test_data[:,i] = X_test_data[:,i] ./ std_feat[i];
    end

    #Center Labels
    mean_train_y = mean(Y_train_data);
    Y_train_data =  Y_train_data .- mean_train_y;
    Y_valid_data = Y_valid_data .- mean_train_y;
    Y_test_data = Y_test_data .- mean_train_y;
    
     
    return(X_train_data, X_valid_data, X_test_data, Y_train_data, Y_valid_data, Y_test_data)
    
end


In [ ]:
# Split the sample dataset so we can test the functions
X_train, X_valid, X_test, Y_train, Y_valid, Y_test = splitter(X_matrix, Y_matrix, 0.5, 0.25);

### MAE function to validate ρ values and evaluate uncertainty sets

In [ ]:
function MAE(X_mat, y_vec, beta_vec)
    return(sum(broadcast(abs, y_vec - X_mat*beta_vec)))
end

### Find the best value of ρ for a robust optimization problem through validation

In [ ]:
function row_tester(row_list, X_train, Y_train, X_valid, Y_valid, func)
    
    MAE_errors = []
    
    # Loop through different possible values for ρ
    # Calculate beta value (training data) and then MAE value (validation data) for that particular ρ
    # Save your MAE values in an array
    for p in row_list
        beta_temp = func(X_train, Y_train, p)
        MAE_tmp = MAE(X_valid, Y_valid, beta_temp)
        append!(MAE_errors, MAE_tmp)
    end
    
    min_index = argmin(MAE_errors)  # find the minimum MAE  
    return(row_list[min_index]) # return the value of ρ that gave the minimum MAE on the validation set

end

### Test all uncertainty sets on a dataset and return the name of the one that resulted in the lowest test MAE

In [ ]:
function best_uncertainty_set(X_train, X_valid, X_test, Y_train, Y_valid, Y_test, row_range)
    # Set up dictionary with starting values (to be replaced)
    sets_dict = Dict([("Frobenius One", 0.5), ("Frobenius Two", 0.5), ("Frobenius Infinity", 0.5),
                  ("Induced One Two", 0.5), ("Induced Two One", 0.5), 
                    ("Induced Two Infinity", 0.5), ("Induced Infinity Two", 0.5)])
    
    # Find best ρ for Frobenius One uncertainty set, and save MAE for Frobenius One beta
    row_frob_one = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, frobenius_one)
    beta_frob_one = frobenius_one(X_train, Y_train, row_frob_one)
    sets_dict["Frobenius One"] = MAE(X_test, Y_test, beta_frob_one)
    
    # Find best ρ for Frobenius Two uncertainty set, and save MAE for Frobenius Two beta
    row_frob_two = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, frobenius_two)
    beta_frob_two = frobenius_two(X_train, Y_train, row_frob_one)
    sets_dict["Frobenius Two"] = MAE(X_test, Y_test, beta_frob_two)
    
    # Find best ρ for Frobenius Infinity uncertainty set, and save MAE for Frobenius Infinity beta
    row_frob_infinity = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, frobenius_infinity)
    beta_frob_infinity = frobenius_infinity(X_train, Y_train, row_frob_infinity)
    sets_dict["Frobenius Infinity"] = MAE(X_test, Y_test, beta_frob_infinity)
    
    # Find best ρ for Induced One Two uncertainty set, and save MAE for Induced One Two beta
    row_induced_12 = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_one_two)
    beta_induced_12 = induced_one_two(X_train, Y_train, row_induced_12)
    sets_dict["Induced One Two"] = MAE(X_test, Y_test, beta_induced_12)
    
    # Find best ρ for Induced Two One uncertainty set, and save MAE for Induced Two One beta
    row_induced_21 = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_two_one)
    beta_induced_21 = induced_two_one(X_train, Y_train, row_induced_21)
    sets_dict["Induced Two One"] = MAE(X_test, Y_test, beta_induced_21)
    
    # Find best ρ for Induced Two Infinity uncertainty set, and save MAE for Induced Two Infinity beta
    row_induced_2inf = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_two_infinity)
    beta_induced_2inf = induced_two_infinity(X_train, Y_train, row_induced_2inf)
    sets_dict["Induced Two Infinity"] = MAE(X_test, Y_test, beta_induced_2inf)
    
    # Find best ρ for Induced Infinity Two uncertainty set, and save MAE for Induced Infinity Two beta
    row_induced_inf2 = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_infinity_two)
    beta_induced_inf2 = induced_infinity_two(X_train, Y_train, row_induced_inf2)
    sets_dict["Induced Infinity Two"] = MAE(X_test, Y_test, beta_induced_inf2)
    
    # Return name of uncertainty set that had t
    return(findmin(sets_dict)[2])
end

### Test only the most common uncertainty sets on a dataset and return the name of the one that resulted in the lowest test MAE

In [ ]:
function best_uncertainty_reduced(X_train, X_valid, X_test, Y_train, Y_valid, Y_test, row_range)
    # Set up dictionary with starting values (to be replaced)
    sets_dict = Dict([("Induced Two One", 0.5), 
                    ("Induced Two Infinity", 0.5), 
            ("Induced Infinity Two", 0.5)])
     
    # Find best ρ for Induced Two One uncertainty set, and save MAE for Induced Two One beta
    row_induced_21 = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_two_one)
    beta_induced_21 = induced_two_one(X_train, Y_train, row_induced_21)
    sets_dict["Induced Two One"] = MAE(X_test, Y_test, beta_induced_21)
    
    # Find best ρ for Induced Two Infinity uncertainty set, and save MAE for Induced Two Infinity beta
    row_induced_2inf = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_two_infinity)
    beta_induced_2inf = induced_two_infinity(X_train, Y_train, row_induced_2inf)
    sets_dict["Induced Two Infinity"] = MAE(X_test, Y_test, beta_induced_2inf)
    
    # Find best ρ for Induced Infinity Two uncertainty set, and save MAE for Induced Infinity Two beta
    row_induced_inf2 = row_tester(row_range, X_train, Y_train, X_valid, Y_valid, induced_infinity_two)
    beta_induced_inf2 = induced_infinity_two(X_train, Y_train, row_induced_inf2)
    sets_dict["Induced Infinity Two"] = MAE(X_test, Y_test, beta_induced_inf2)
    
    # Return name of uncertainty set that had t
    return(findmin(sets_dict)[2])
end

### Build a function that returns features for the Optimal Classification tree to split upon
#### Features: $R^2$, domain, number of features, number of observations, normalized variance of the labels, avg normalzied variance of the features, median normalized variance of the features, range of the normalized variance of the features, percentage of outliers in the labels, percent of features that have at least some threshold of correlation, percent of missing labels, avg percent of missing values in the features, mean correlation (fisher's z-test).

In [ ]:
# Create an empty features dataframe
meta_df = DataFrame(
    Dataset_Name = String[], 
    Source = String[],
    y_miss_per = Float64[],
    avg_per_miss_x = Float64[],
    r2_rr = Float64[], 
    n = Int64[], 
    p = Int64[],
    ind_Disp_y = Float64[], 
    x_mean_norm_var = Float64[],
    median_x_norm_var = Float64[],
    range_norm_x_var = Float64[], 
    y_outliers_per = Float64[],
    x_corr_per = Float64[], 
    mean_cor = Float64[],
    uncertainty_set = String[])

### Get list of filtered R Datasets
### We filtered out certain R packages and datasets with fewer than 40 rows

In [ ]:
RDataset_List = CSV.read("RDatasets_Filtered.csv");

### Loop through the R datasets, generate features, and add optimal uncertainty set label

#### Function to impute missing data with k-nearest-neighbors

In [ ]:
function knn_impute(dataset)
    # Convert NaN to missing
    for col in 1:size(dataset, 2)
        replace(dataset[!,col], NaN => missing)
    end
    
    # Cross validation
    grid = IAI.GridSearch(IAI.ImputationLearner(random_seed=2),
        (method=:opt_knn, knn_k = [5:5:100;]))
    best_mod = IAI.get_learner(grid)

    IAI.fit!(best_mod,dataset)
    data_imputed = IAI.transform(best_mod, dataset)
    return(data_imputed)
end

#### Function to split a dataset into X and y labels

In [ ]:
function features(data)
    data = convert(Matrix,data)
    X = data[:,1:end-1]
    y = data[:,end]
    
    # split data
    X_train, X_valid, X_test, Y_train, Y_valid, Y_test = splitter(X,y,0.5,0.25)
    
    r2_rr = R2_RR(X_train, X_valid, X_test, Y_train, Y_valid, Y_test) ##returns r^2 for ridge regression
    
    n = size(X,1)
    p = size(X,2)
    
    if mean(y) == 0 && var(y) == 0
        ind_Disp_y = 0
    else
        ind_Disp_y = var(y)/mean(y)
    end
    
    x_mean_norm_var, x_norm_var = mean_norm_var(X)
    
    #mean normalized variance of the features
    median_x_norm_var = median(x_norm_var)

    range_norm_x_var = maximum(x_norm_var)-minimum(x_norm_var)
    
    y_outliers_per = count(i-> -2*std(y)<i<2*std(y), y)/length(y)
    
    #mean_cor = mean(cor(X)) this is wrong!
    #percentage of features that are correlated above a certain threshold
    x_corr_per = 1- count(i-> (-0.4<i<0.4),cor(X))/(length(cor(X))-p) 
    
    ## vectorize correlation matrix. z transform each element, then average, then reverse transform. 
    ## This approach requires some assumptions on the data
    mean_cor = correlation_metric(X,p)
    
    uncertainty_set = best_uncertainty_reduced(X_train, X_valid, X_test, Y_train, Y_valid, Y_test, collect(0:0.5:10))
     
    return [r2_rr, n, p, 
        ind_Disp_y, x_mean_norm_var, median_x_norm_var,
        range_norm_x_var, y_outliers_per, x_corr_per, mean_cor, uncertainty_set]      
end


### Feature generation functions

In [ ]:
function perc_missing_x(X)
    x_miss = []
    for j in 1:size(X,2)
        append!(x_miss, count(i -> typeof(i)==Missing, X[:,j])/length(X[:,j]))
    end
    return mean(x_miss)
end

In [ ]:
function perc_missing_y(Y)
    return (count(i -> typeof(i)==Missing, Y[:,1])/size(Y, 1))
end

In [ ]:
function mean_norm_var(X)
    x_norm_var = []
    for i in 1:size(X,2)
        if mean(X[:,i]) == 0
            append!(x_norm_var, 0)
        else
            append!(x_norm_var, var(X[:,i])/mean(X[:,i]))
        end
    end
    return mean(x_norm_var), x_norm_var #mean normalized variance of the features
end
    

In [ ]:
function correlation_metric(X,p)
    vec_cor =[]
    mat_cor = cor(X)
    for i in 1:p
        for j in 1:p
            if j>i
                append!(vec_cor,mat_cor[i,j])
            end
        end
    end
    return tanh(mean(atanh.(vec_cor)))
end

#### Ridge regression for calculating the β's to find an estimate $\hat{y}$

In [ ]:
function L2(train_X,train_y,ρ)
    m = Model(solver=GurobiSolver(PSDTol=1e-3))
    
    P = size(train_X)[2]
    n = size(train_X)[1]
    
    @variable(m, z[1:2])
    @variable(m, β[1:P])


    @constraint(m, z[1] >= sum((train_y[i] - transpose(β)*train_X[i,:])^2 for i in 1:n))
    @constraint(m, z[2] >= sum(β[i]^2 for i in 1:P))

    
    @objective(m, Min, z[1] + ρ*z[2])
    
    solve(m)
    
    return getvalue(β)
end

#### Adaption of earlier function that uses MSE instead to judge the validation loss because ridge regression optimimizes on squared error

In [ ]:
function row_tester_MSE(row_list, X_train, Y_train, X_valid, Y_valid, func)
    
    MSE_errors = []
    βs = []
    
    # Loop through different possible values for ρ
    # Calculate beta value (training data) and then MAE value (validation data) for that particular ρ
    # Save your MSE values in an array
    for p in row_list
        beta_temp = func(X_train, Y_train, p)
        push!(βs,beta_temp)
        MSE_tmp = MSE(X_valid, Y_valid, beta_temp)
        append!(MSE_errors, MSE_tmp)
    end
    
    min_index = argmin(MSE_errors)  # find the index of that minimum MSE
    return(βs[min_index]) # return the value of β that gave the minimum MSE on the validation set

end

In [ ]:
function MSE(X_valid,Y_valid,beta_temp)
    return sum((Y_valid[i] - transpose(beta_temp)*X_valid[i,:])^2 for i=1:size(X_valid,1))
end

In [ ]:
function R2_RR(X_train, X_valid, X_test, Y_train, Y_valid, Y_test) #returns a different value for R^2 because of randomization of the splits
    #We want to use all the data, because we only care about finding ρ to generate an unbiased $R^2$
    opt_β = row_tester_MSE([0.1,1,5,10,20],X_train,Y_train,X_valid,Y_valid,L2);
    y_hat = X_test*opt_β
    mean_y = mean(Y_test)
    y_bar = fill(mean_y,size(Y_test,1))
    return 1 - sum((Y_test .- y_hat).^2)/sum((Y_test .- y_bar).^2)
end


If we are going to use $R^2$ as a feature, we must understand what it is doing so that we can interpret the OCT results. One possible interpretation is that $R^2$ guages how amenable a dataset is to a linear model. But to calculate, we need estimates, $\hat{y}$, that come from β. Estimating the β requires some approach, and statistical learning theory shows us that different approaches have different biases. Since our  metric for evaluating performance on the regressions is Ordinary Least Squares, $||Y-Xβ||^2$ is the natural loss function to minimize to recover the β. If the system is overdetermined, (n>d), there is not guaranteed to exist a β that satisfies $Y = Xβ$. The normal equations yield the β that minimizes squared loss-- $β=(X^TX)^{-1}X^TY$. But if $d >>n $, then taking the inverse of $X^TX$ becomes less computationally feasible. If the system is underdetermined, n<d, the solution is possibly not unique. 

One approach in calculating $R^2$ for the purposes of classifying the datasets is to use 1) $β=(X^TX)^{-1}X^T$ if n>d, and 2) $β=X^T(XX^T)^{-1}$ if n<d. The tradeoff is that while 1) minimizes squared loss, 2) minimizes the squared sum of the components of β s.t. $Y=Xβ$. So the predictor would be arbitrarily biased depending on whether n>d or n<d.

Another approach to calculate $R^2$ for the purposes of classifying the datasets is to use Gurobi to solve ridge regression, because we know that this scales for large datasets. We can then interpret the Optimal Classification Tree results. We use the latter for the feature included in Optimal Classification Tree. We provide a range for ρ and select the best via cross validation. 

### Impute missing data

In [ ]:
for i in 1:size(RDataset_List, 1)
    name = RDataset_List[i,:Dataset] # name of dataset to read in
    source = RDataset_List[i,:Package] # source of dataset to read in
    print(name, " ", source, "\n")
    tmp_df = dataset(source, name) # read in datasets one at a time
    
    
    tmp_features = Any[name, source] # Create a list of features to add to meta_df
    
    # Find missing data points
    y_miss_per = perc_missing_y(tmp_df[!,end])
    avg_per_miss_x = perc_missing_x(convert(Matrix, tmp_df[!,1:end-1]))
    append!(tmp_features, y_miss_per)
    append!(tmp_features, avg_per_miss_x)
    
    # Get a list of the column types in tmp_df
    col_types = []
    for n in names(tmp_df)
        push!(col_types, typeof(tmp_df[!,n]))     
    end
    
    # Skip datasets with non-numeric values
    non_numeric = false
    for t in col_types
        if occursin("Categorical", string(t))
            non_numeric = true
        elseif occursin("String", string(t))
            non_numeric = true
        end
    end
    
    if non_numeric == true
        continue
    end
    
    tmp_df = knn_impute(tmp_df) # Impute any missing data  
    
    print(name, " ", source, " " , i, "\n")
        
    features(tmp_df)
    append!(tmp_features, features(tmp_df))
    
    push!(meta_df, tmp_features)     
end

### Save metadata as a csv

In [ ]:
CSV.write("Final_Metadata_Reduced.csv", meta_df);